In [11]:
import os
import time
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Obtenha o diretório atual do script Python
script_dir = os.getcwd()

# Configure as opções do Chrome para o modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# Configurar o diretório de download para um local específico
download_dir = os.path.join(script_dir, 'diarios')  # Diretório de download relativo ao diretório do script
shutil.rmtree(download_dir, ignore_errors=True)
os.makedirs(download_dir)
prefs = {'download.default_directory': download_dir}
chrome_options.add_experimental_option('prefs', prefs)

# Configurar o serviço do ChromeDriver
chrome_service = ChromeService(ChromeDriverManager().install())

# Inicializar o navegador Chrome
nav = webdriver.Chrome(service=chrome_service, options=chrome_options)
nav.get("https://www.diariomunicipal.com.br/ama/")

# Interagir com o site para fazer o download
nav.find_element(By.CLASS_NAME, "selected").click()
elemento = WebDriverWait(nav, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="btDownloadSimples2"]')))

# Obter o nome original do arquivo da URL
url = elemento.get_attribute("href")
nome_original = url.split("publicado_")[-1]
nome_original = "publicado_" + nome_original

# Caminho local completo para o arquivo baixado
caminho_local_download = os.path.join(download_dir, nome_original)

# Clicar para iniciar o download
elemento.click()

# Esperar até que o arquivo seja completamente baixado
while not os.path.exists(caminho_local_download):
    time.sleep(1)

# Configurar autenticação automática com o Google Drive usando OAuth2
gauth = GoogleAuth()
#gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# Verificar se o arquivo "today" antigo existe no Google Drive e excluí-lo
file_list = drive.ListFile({'q': "title = 'today.pdf'"}).GetList()
if file_list:
    old_file = file_list[0]
    old_file.Delete()  # Excluir o arquivo antigo

# Verificar se o arquivo existe antes de fazer upload
if os.path.exists(caminho_local_download):
    # Fazer upload do arquivo renomeado para o Google Drive
    arquivo = drive.CreateFile({'title': 'today.pdf'})  # Nome do arquivo no Google Drive
    arquivo.SetContentFile(caminho_local_download)  # Especifique o caminho local do arquivo a ser enviado
    arquivo.Upload()
    print(f'Arquivo {arquivo["title"]} foi enviado para o Google Drive com sucesso.')
else:
    print(f'O arquivo {nome_original} não foi encontrado no caminho especificado.')

# Fechar o navegador após o download e envio do arquivo
nav.quit()


Arquivo today.pdf foi enviado para o Google Drive com sucesso.
